In [80]:
using DataFrames, CSV, MLDataUtils, ScikitLearn, CategoricalArrays

In [ ]:
@sk_import metrics: (accuracy_score, confusion_matrix, classification_report)

In [218]:
final_table = CSV.read("final_table5.csv", DataFrame, missingstring="?")
dropmissing!(final_table)

Row,Date,Flight Number,Carrier Code,Destination Airport,Scheduled departure time,Maximum,Minimum,Temperature,Precipitation,SecurityWaitTime,BaggageClaimTime,Day of Week,Visibility,Delayed Minutes,Is Delayed,TreatmentTime,Outcome,AdjustedOutcome,EstimatedTrafficNumeric
,String7,Int64,Int64,Int64,Time,Int64,Int64,Float64,Float64,Float64,Float64,Int64,Float64,Int64,Int64,Int64,Float64,Float64,Int64
1,3/1/23,334,2,0,15:20:00,42,33,37.5,0.0,18.45,20.7,2,5.48818,1,1,63,-7.15,82.85,0
2,3/1/23,345,2,45,07:05:00,42,33,37.5,0.0,28.5,8.85,2,5.54049,0,0,62,-5.35,84.65,1
3,3/1/23,363,2,0,12:20:00,42,33,37.5,0.0,14.55,13.8,2,9.47355,3,1,62,0.65,0.65,0
4,3/1/23,365,2,46,07:35:00,42,33,37.5,0.0,25.65,8.85,2,8.18748,0,0,73,8.5,8.5,1
5,3/1/23,366,2,42,16:30:00,42,33,37.5,0.0,25.65,7.95,2,7.93649,2,1,77,11.4,11.4,1
6,3/1/23,376,2,0,14:05:00,42,33,37.5,0.0,12.0,10.5,2,8.47586,0,0,72,19.5,19.5,0
7,3/1/23,385,2,0,17:15:00,42,33,37.5,0.0,25.65,17.55,2,6.01188,0,0,80,6.8,6.8,1
8,3/1/23,388,2,36,18:20:00,42,33,37.5,0.0,15.9,13.8,2,5.23971,0,0,67,7.3,7.3,1
9,3/1/23,425,2,23,07:25:00,42,33,37.5,0.0,28.5,8.85,2,5.84115,1,1,69,0.65,0.65,1


## Baseline model

In [311]:
using DecisionTree
using DataFrames
using Random
using MLDataUtils

# Set the seed for reproducibility
Random.seed!(12345)

# Assuming you have a DataFrame `final_table` with your data
features = ["Precipitation", "Visibility", "Day of Week", "Temperature", "Destination Airport"]
label_column = "Delayed Minutes"

# Convert DataFrame columns to matrices/vectors
feature_matrix = Matrix(final_table[:, features])
label_vector = Vector{Float64}(final_table[:, label_column])  # Ensure labels are Float64

# Shuffle the observations
indices = shuffleobs(collect(eachindex(label_vector)))

# Split the data into training and testing sets
train_indices, test_indices = splitobs(indices, at = 0.8)
train_features, test_features = feature_matrix[train_indices, :], feature_matrix[test_indices, :]
train_labels, test_labels = label_vector[train_indices], label_vector[test_indices]

# Initialize and fit the model
model = build_tree(train_labels, train_features)

# Apply learned model to make predictions (example prediction)
#prediction = apply_tree(model, test_features)

# Run 3-fold cross-validation, returns array of coefficients of determination (R^2)
n_folds = 3
r2 = nfoldCV_tree(train_labels, train_features, n_folds)

println("R^2 from cross-validation: $r2")

print_tree(model)


Fold 1
Mean Squared Error:     17.153765447458735
Correlation Coeff:      0.18202627771214167
Coeff of Determination: -0.18325707607113562

Fold 2
Mean Squared Error:     17.65482454258997
Correlation Coeff:      0.15903430968254859
Coeff of Determination: -0.25696008179608

Fold 3
Mean Squared Error:     17.19990522819654
Correlation Coeff:      0.15572589522791824
Coeff of Determination: -0.2442829414771428

Mean Coeff of Determination: -0.2281666997814528
R^2 from cross-validation: [-0.18325707607113562, -0.25696008179608, -0.2442829414771428]
Feature 1 < 0.275 ?
├─ Feature 5 < 25.5 ?
    ├─ Feature 5 < 2.5 ?
        ├─ Feature 5 < 0.5 ?
            ├─ Feature 3 < 1.5 ?
                ├─ Feature 4 < 40.75 ?
                    ├─ Feature 4 < 37.25 ?
                        ├─ Feature 2 < 6.4 ?
                            ├─ 2.6 : 0/5
                            └─ 3.3333333333333335 : 0/6
                        └─ Feature 2 < 6.067 ?
                            ├─ 1.0 : 0/5
     

## OCT for Predicting Flight Delay

In [305]:
features = ["Precipitation", "Visibility", "Day of Week", "Temperature", "Destination Airport"]  # All feature names as strings
target = "Delayed Minutes"  

X = final_table[:, features]
y = final_table[:, target]

(train_X, train_y), (test_X, test_y) = IAI.split_data(:regression, X, y,
                                                      seed=12345)
 

# Initialize an Optimal Classification Tree classifier
grid = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=123,
    ),
    max_depth=1:5,
)
IAI.fit!(grid, train_X, train_y)
IAI.get_learner(grid)

Fitted OptimalTreeRegressor:
  1) Split: Precipitation < 0.275
    2) Split: Destination Airport < 23.5
      3) Split: Destination Airport < 8.5
        4) Split: Destination Airport < 2.5
          5) Predict: 1.971, 407 points, error 7.93
          6) Predict: 3.776, 424 points, error 15.12
        7) Split: Destination Airport < 10.5
          8) Predict: 5.557, 456 points, error 17.84
          9) Predict: 3.321, 1506 points, error 13.91
      10) Split: Destination Airport < 25.5
        11) Split: Temperature < 39.5
          12) Predict: 4.208, 101 points, error 19.41
          13) Predict: 6.5, 136 points, error 17.32
        14) Split: Destination Airport < 27.5
          15) Predict: 1.771, 315 points, error 7.846
          16) Predict: 3.065, 2075 points, error 14.06
    17) Split: Day of Week < 4.5
      18) Split: Destination Airport < 10.5
        19) Split: Destination Airport < 6.5
          20) Predict: 1.507, 136 points, error 7.118
          21) Predict: 4.364, 129 points, error 15.72
        22) Split: Precipitation < 0.33
          23) Predict: 3.073, 165 points, error 15.93
          24) Predict: 2.03, 691 points, error 9.335
      25) Predict: 0.7619, 189 points, error 3.758

In [306]:
IAI.predict(grid, test_X)

2884-element Vector{Float64}:
 1.970515970515969
 3.3207171314741135
 1.970515970515969
 1.970515970515969
 4.2079207920792046
 3.06506024096387
 3.3207171314741135
 1.970515970515969
 3.06506024096387
 4.2079207920792046
 1.970515970515969
 4.2079207920792046
 3.3207171314741135
 ⋮
 3.7759433962264186
 5.557017543859654
 5.557017543859654
 5.557017543859654
 3.3207171314741135
 3.7759433962264186
 3.06506024096387
 6.4999999999999964
 3.06506024096387
 6.4999999999999964
 3.3207171314741135
 3.3207171314741135

In [307]:
IAI.score(grid, train_X, train_y, criterion=:mse)

0.08539026322590348

In [308]:
IAI.score(grid, test_X, test_y, criterion=:mse)

0.06595783615314843

## OCT with Hyperplanes

In [313]:
grid = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=12345,
        hyperplane_config=(sparsity=:all,)
    ),
    max_depth=1:4,
)
IAI.fit!(grid, train_X, train_y)
IAI.get_learner(grid)

Fitted OptimalTreeRegressor:
  1) Split: 0.1131 * Precipitation - 0.004598 * Temperature + 0.02044 * Destination Airport < 0.08404
    2) Split: Destination Airport < 8.5
      3) Split: Destination Airport < 2.5
        4) Split: Destination Airport < 0.5
          5) Predict: 1.34, 300 points, error 5.298
          6) Predict: 2.465, 202 points, error 9.932
        7) Split: Temperature < 34.25
          8) Predict: 0.3077, 13 points, error 1.136
          9) Predict: 3.722, 493 points, error 14.6
      10) Split: Destination Airport < 10.5
        11) Split: Day of Week < 4.5
          12) Predict: 5.861, 410 points, error 17.94
          13) Predict: 4.17, 100 points, error 14.22
        14) Split: Temperature < 43.25
          15) Predict: 2.312, 237 points, error 8.527
          16) Predict: 4.24, 200 points, error 14.93
    17) Split: 0.04339 * Precipitation - 0.01455 * Temperature < -0.5728
      18) Split: Destination Airport < 25.5
        19) Split: Destination Airport < 23.5
          20) Predict: 3.354, 599 points, error 14.52
          21) Predict: 6.356, 146 points, error 17.64
        22) Split: Destination Airport < 27.5
          23) Predict: 1.684, 187 points, error 7.029
          24) Predict: 3.357, 1255 points, error 15.16
      25) Split: 60.48 * Precipitation + 0.0924 * Day of Week + 0.005705 * Destination Airport < 0.4158
        26) Predict: 3.922, 603 points, error 17.03
        27) Split: Temperature < 34.5
          28) Predict: 0.8199, 161 points, error 4.086
          29) Predict: 2.231, 1824 points, error 10.35

In [288]:
IAI.score(grid, test_X, test_y, criterion=:mse)

0.06418268645933867

## Optimal Policy Trees

In [220]:
final_table

Row,Date,Flight Number,Carrier Code,Destination Airport,Scheduled departure time,Maximum,Minimum,Temperature,Precipitation,SecurityWaitTime,BaggageClaimTime,Day of Week,Visibility,Delayed Minutes,Is Delayed,TreatmentTime,Outcome,AdjustedOutcome,EstimatedTrafficNumeric
,String7,Int64,Int64,Int64,Time,Int64,Int64,Float64,Float64,Float64,Float64,Int64,Float64,Int64,Int64,Int64,Float64,Float64,Int64
1,3/1/23,334,2,0,15:20:00,42,33,37.5,0.0,18.45,20.7,2,5.48818,1,1,63,-7.15,82.85,0
2,3/1/23,345,2,45,07:05:00,42,33,37.5,0.0,28.5,8.85,2,5.54049,0,0,62,-5.35,84.65,1
3,3/1/23,363,2,0,12:20:00,42,33,37.5,0.0,14.55,13.8,2,9.47355,3,1,62,0.65,0.65,0
4,3/1/23,365,2,46,07:35:00,42,33,37.5,0.0,25.65,8.85,2,8.18748,0,0,73,8.5,8.5,1
5,3/1/23,366,2,42,16:30:00,42,33,37.5,0.0,25.65,7.95,2,7.93649,2,1,77,11.4,11.4,1
6,3/1/23,376,2,0,14:05:00,42,33,37.5,0.0,12.0,10.5,2,8.47586,0,0,72,19.5,19.5,0
7,3/1/23,385,2,0,17:15:00,42,33,37.5,0.0,25.65,17.55,2,6.01188,0,0,80,6.8,6.8,1
8,3/1/23,388,2,36,18:20:00,42,33,37.5,0.0,15.9,13.8,2,5.23971,0,0,67,7.3,7.3,1
9,3/1/23,425,2,23,07:25:00,42,33,37.5,0.0,28.5,8.85,2,5.84115,1,1,69,0.65,0.65,1


In [236]:
treatments = final_table.TreatmentTime
outcomes = final_table.AdjustedOutcome

# Exclude the treatments and outcome columns from X
X = select(final_table, Not(["TreatmentTime", "AdjustedOutcome","Outcome", "Date", "Scheduled departure time", "Is Delayed", "Flight Number", "Maximum", "Minimum"]))

(train_X, train_treatments, train_outcomes), (test_X, test_treatments, test_outcomes) =
    IAI.split_data(:policy_maximize, X, treatments, outcomes, seed=123, train_proportion=0.5)

((4807×10 DataFrame
  Row │ Carrier Code  Destination Airport  Temperature  Precipitation  Securit ⋯
      │ Int64         Int64                Float64      Float64        Float64 ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │            2                    0         37.5           0.0           ⋯
    2 │            2                   46         37.5           0.0
    3 │            2                    0         37.5           0.0
    4 │            2                   36         37.5           0.0
    5 │            2                   12         37.5           0.0           ⋯
    6 │            2                    0         37.5           0.0
    7 │            2                   23         37.5           0.0
    8 │            2                   33         37.5           0.0
    9 │            2                   52         37.5           0.0           ⋯
   10 │            2                   11         37.5           0.0
   11 │    

In [240]:
treatment_candidates =60:5:90

60:5:90

In [241]:
using Statistics

# Count the frequency of each treatment candidate in train_treatments
for treatment in 60:5:90
    count = sum(train_treatments .== treatment)
    println("Treatment $treatment: $count samples")
end


Treatment 60: 155 samples
Treatment 65: 162 samples
Treatment 70: 168 samples
Treatment 75: 139 samples
Treatment 80: 158 samples
Treatment 85: 145 samples
Treatment 90: 150 samples


In [242]:
reward_lnr = IAI.NumericRegressionRewardEstimator(
    propensity_estimator=IAI.RandomForestRegressor(),
    outcome_estimator=IAI.RandomForestRegressor(),
    reward_estimator=:doubly_robust,
    propensity_min_value=0.1,
    random_seed=1,
)

train_predictions, train_reward_score = IAI.fit_predict!(
    reward_lnr, train_X, train_treatments, train_outcomes, treatment_candidates)
train_rewards = train_predictions[:reward]

Row,60,65,70,75,80,85,90
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-155.888,1.3789,6.35313,14.3438,18.3961,23.2888,27.2432
2,77.9391,38.0727,9.51267,4.62255,13.644,19.986,22.0261
3,7.55217,12.7629,27.9714,18.5474,28.099,31.7581,36.0058
4,27.2949,6.5754,7.38222,14.2544,19.6689,25.6378,28.2604
5,36.8335,18.333,9.47655,12.9031,25.5394,19.0111,30.0845
6,80.9396,23.5046,7.97092,10.5046,19.8624,16.3903,23.1859
7,72.1786,82.7322,24.7256,10.6053,-5.46397,15.3555,18.3286
8,26.9019,27.4351,32.2375,37.297,38.6717,44.7852,48.0937
9,411.499,3.09679,9.25435,13.0885,18.628,23.3085,27.1817


In [243]:
train_rewards = train_predictions[:reward]

Row,60,65,70,75,80,85,90
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-155.888,1.3789,6.35313,14.3438,18.3961,23.2888,27.2432
2,77.9391,38.0727,9.51267,4.62255,13.644,19.986,22.0261
3,7.55217,12.7629,27.9714,18.5474,28.099,31.7581,36.0058
4,27.2949,6.5754,7.38222,14.2544,19.6689,25.6378,28.2604
5,36.8335,18.333,9.47655,12.9031,25.5394,19.0111,30.0845
6,80.9396,23.5046,7.97092,10.5046,19.8624,16.3903,23.1859
7,72.1786,82.7322,24.7256,10.6053,-5.46397,15.3555,18.3286
8,26.9019,27.4351,32.2375,37.297,38.6717,44.7852,48.0937
9,411.499,3.09679,9.25435,13.0885,18.628,23.3085,27.1817


In [244]:
train_reward_score[:propensity]

Dict{String, Float64} with 7 entries:
  "80" => -0.0180228
  "75" => -0.0311084
  "85" => -0.0333148
  "70" => -0.0298613
  "90" => -0.0381644
  "60" => -0.0481185
  "65" => -0.0245706

In [245]:
train_reward_score[:outcome]

Dict{String, Float64} with 7 entries:
  "80" => 0.700169
  "75" => 0.629417
  "85" => 0.713581
  "70" => 0.687014
  "90" => 0.675614
  "60" => 0.735687
  "65" => 0.714331

In [246]:
grid = IAI.GridSearch(
    IAI.OptimalTreePolicyMinimizer(
        random_seed=1,
        minbucket=10,
    ),
    max_depth=2:7,
)
IAI.fit!(grid, train_X, train_rewards)

All Grid Results:

 Row │ max_depth  cp           train_score  valid_score  rank_valid_score
     │ Int64      Float64      Float64      Float64      Int64
─────┼────────────────────────────────────────────────────────────────────
   1 │         2  0.00111199     0.0111991    0.0128525                 6
   2 │         3  0.000662164    0.0157679    0.017683                  5
   3 │         4  4.63159e-5     0.0188463    0.0200827                 1
   4 │         5  6.39621e-5     0.0208134    0.0192306                 2
   5 │         6  0.000160464    0.0233758    0.0191008                 3
   6 │         7  2.81972e-5     0.0251394    0.0188448                 4

Best Params:
  cp => 4.631585099873492e-5
  max_depth => 4

Best Model - Fitted OptimalTreePolicyMinimizer:
  1) Split: SecurityWaitTime < 17.7
    2) Split: Delayed Minutes < 3.5
      3) Split: SecurityWaitTime < 15.52
        4) Split: Delayed Minutes < 1.5
          5) Prescribe: 60, 1001 points, error 747.6
          6) Prescribe: 65, 192 points, error 750
        7) Split: Day of Week < 2.5
          8) Prescribe: 70, 158 points, error 746.3
          9) Prescribe: 65, 227 points, error 739.7
      10) Split: SecurityWaitTime < 9.075
        11) Split: Delayed Minutes < 11.5
          12) Prescribe: 60, 382 points, error 752.2
          13) Prescribe: 70, 18 points, error 743.5
        14) Split: Delayed Minutes < 7.5
          15) Prescribe: 70, 329 points, error 744.6
          16) Prescribe: 75, 239 points, error 746.6
    17) Split: BaggageClaimTime < 11.25
      18) Split: Delayed Minutes < 2.5
        19) Split: SecurityWaitTime < 24.98
          20) Prescribe: 65, 330 points, error 744.1
          21) Prescribe: 70, 475 points, error 745.4
        22) Split: Delayed Minutes < 8.5
          23) Prescribe: 75, 492 points, error 745.5
          24) Prescribe: 85, 177 points, error 750.1
      25) Split: SecurityWaitTime < 29.93
        26) Split: Delayed Minutes < 3.5
          27) Prescribe: 80, 360 points, error 746.4
          28) Prescribe: 90, 201 points, error 745.6
        29) Split: Delayed Minutes < 6.5
          30) Prescribe: 90, 191 points, error 747.2
          31) Prescribe: 60, 35 points, error 797.8

In [247]:
prescriptions = IAI.predict(grid, train_X)


4807-element Vector{String}:
 "65"
 "70"
 "60"
 "70"
 "70"
 "70"
 "80"
 "60"
 "65"
 "70"
 "75"
 "60"
 "70"
 ⋮
 "90"
 "75"
 "75"
 "60"
 "65"
 "85"
 "60"
 "85"
 "90"
 "60"
 "75"
 "80"

In [248]:
IAI.convert_treatments_to_numeric(prescriptions)

4807-element Vector{Int64}:
 65
 70
 60
 70
 70
 70
 80
 60
 65
 70
 75
 60
 70
  ⋮
 90
 75
 75
 60
 65
 85
 60
 85
 90
 60
 75
 80

In [249]:
IAI.predict_treatment_outcome(grid, train_X)

Row,60,65,70,75,80,85,90
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,25.4156,11.6995,18.6518,24.1835,28.4989,34.4641,37.9434
2,76.4783,47.2589,7.07518,9.46111,15.037,19.7883,24.148
3,9.25497,13.4228,18.2179,23.3588,27.9229,33.0513,37.2599
4,59.0412,19.6748,7.9255,13.3554,19.1833,22.4932,26.9367
5,76.4783,47.2589,7.07518,9.46111,15.037,19.7883,24.148
6,76.4783,47.2589,7.07518,9.46111,15.037,19.7883,24.148
7,78.6654,83.3756,53.0556,19.9967,8.05508,12.9958,17.1578
8,13.8716,19.8194,23.8425,29.0265,33.7421,39.0265,42.5487
9,60.9346,5.7154,8.8048,13.2207,18.758,23.1822,28.0976


In [250]:
rank = IAI.predict_treatment_rank(grid, train_X)

4807×7 Matrix{String}:
 "65"  "70"  "75"  "60"  "80"  "85"  "90"
 "70"  "75"  "80"  "85"  "90"  "65"  "60"
 "60"  "65"  "70"  "75"  "80"  "85"  "90"
 "70"  "75"  "80"  "65"  "85"  "90"  "60"
 "70"  "75"  "80"  "85"  "90"  "65"  "60"
 "70"  "75"  "80"  "85"  "90"  "65"  "60"
 "80"  "85"  "90"  "75"  "70"  "60"  "65"
 "60"  "65"  "70"  "75"  "80"  "85"  "90"
 "65"  "70"  "75"  "80"  "85"  "90"  "60"
 "70"  "75"  "80"  "85"  "90"  "65"  "60"
 "75"  "80"  "85"  "90"  "70"  "65"  "60"
 "60"  "65"  "70"  "75"  "80"  "85"  "90"
 "70"  "75"  "80"  "65"  "85"  "90"  "60"
 ⋮                             ⋮     
 "90"  "85"  "60"  "80"  "65"  "70"  "75"
 "75"  "80"  "85"  "90"  "70"  "65"  "60"
 "75"  "80"  "85"  "90"  "70"  "65"  "60"
 "60"  "65"  "70"  "75"  "80"  "85"  "90"
 "65"  "70"  "75"  "80"  "85"  "90"  "60"
 "85"  "80"  "90"  "75"  "70"  "60"  "65"
 "60"  "65"  "70"  "75"  "80"  "85"  "90"
 "85"  "80"  "90"  "75"  "70"  "60"  "65"
 "90"  "85"  "80"  "75"  "60"  "65"  "70"
 "60"  "65"  "7

In [251]:
IAI.convert_treatments_to_numeric(rank)

4807×7 Matrix{Int64}:
 65  70  75  60  80  85  90
 70  75  80  85  90  65  60
 60  65  70  75  80  85  90
 70  75  80  65  85  90  60
 70  75  80  85  90  65  60
 70  75  80  85  90  65  60
 80  85  90  75  70  60  65
 60  65  70  75  80  85  90
 65  70  75  80  85  90  60
 70  75  80  85  90  65  60
 75  80  85  90  70  65  60
 60  65  70  75  80  85  90
 70  75  80  65  85  90  60
  ⋮                   ⋮  
 90  85  60  80  65  70  75
 75  80  85  90  70  65  60
 75  80  85  90  70  65  60
 60  65  70  75  80  85  90
 65  70  75  80  85  90  60
 85  80  90  75  70  60  65
 60  65  70  75  80  85  90
 85  80  90  75  70  60  65
 90  85  80  75  60  65  70
 60  65  70  75  80  85  90
 75  80  85  90  70  65  60
 80  85  90  75  70  60  65

In [252]:
IAI.predict_treatment_outcome_standard_error(grid, train_X)

Row,60,65,70,75,80,85,90
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,6.25681,1.81697,0.819236,0.838069,0.825748,0.810323,0.834235
2,2.67789,4.21497,1.33662,0.462648,0.21846,0.202208,0.261272
3,0.871265,0.312514,0.292902,0.293884,0.286553,0.296376,0.320666
4,11.012,5.26063,0.376553,0.633178,0.36221,0.376017,0.310559
5,2.67789,4.21497,1.33662,0.462648,0.21846,0.202208,0.261272
6,2.67789,4.21497,1.33662,0.462648,0.21846,0.202208,0.261272
7,1.14235,0.430492,3.70741,4.57836,0.502746,0.260097,0.356564
8,1.69341,0.64041,0.625368,0.519579,0.510352,0.527072,0.625332
9,6.55721,2.91958,0.333985,0.266325,0.24892,0.211163,0.316746


In [253]:
test_predictions, test_reward_score = IAI.fit_predict!(
    reward_lnr, test_X, test_treatments, test_outcomes, treatment_candidates)
test_rewards = test_predictions[:reward]

Row,60,65,70,75,80,85,90
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,88.8841,91.6225,75.662,14.5873,12.973,15.3327,16.9268
2,137.999,90.5022,5.18513,9.52339,12.3307,18.1392,21.8842
3,77.7998,43.259,6.97881,20.104,11.2389,20.09,21.3239
4,76.4138,79.9484,80.5546,8.78644,3.42726,11.9275,15.4449
5,84.2292,80.3438,-147.353,9.04452,11.1064,16.9905,20.8265
6,10.987,0.637375,11.1771,13.9172,20.5095,26.4418,27.8781
7,18.5902,6.44003,9.86428,14.4679,19.45,28.5319,26.4489
8,5.44319,8.63253,12.7574,16.682,22.297,24.0124,30.1126
9,20.7685,20.5216,27.4389,25.1748,33.972,41.2249,44.3358


In [254]:
test_reward_score[:propensity]

Dict{String, Float64} with 7 entries:
  "80" => -0.0238481
  "75" => -0.0247529
  "85" => -0.0197991
  "70" => -0.0324936
  "90" => -0.0282164
  "60" => -0.0387121
  "65" => -0.0323058

In [255]:
test_reward_score[:outcome]

Dict{String, Float64} with 7 entries:
  "80" => 0.702739
  "75" => 0.577457
  "85" => 0.722775
  "70" => 0.663802
  "90" => 0.754628
  "60" => 0.757697
  "65" => 0.637712

In [314]:
prescriptions = IAI.predict(grid, test_X)

2884-element Vector{Float64}:
 1.3399999999999987
 3.9220563847429557
 1.3399999999999987
 1.3399999999999987
 3.9220563847429557
 2.231359649122814
 3.9220563847429557
 1.3399999999999987
 2.231359649122814
 3.9220563847429557
 2.4653465346534627
 3.9220563847429557
 3.9220563847429557
 ⋮
 3.7221095334685637
 5.860975609756102
 5.860975609756102
 5.860975609756102
 3.3539232053422436
 3.7221095334685637
 3.3569721115537874
 6.356164383561641
 3.3569721115537874
 6.356164383561641
 3.3539232053422436
 3.3539232053422436

In [316]:
using ScikitLearn
@sk_import metrics:roc_auc_score

[ Info: mkl not found, proceeding to installing non-mkl versions of sci-kit learn via Conda
[ Info: Running `conda install -y -c conda-forge 'scikit-learn>=1.2,<1.3'` in root environment


Retrieving notices: ...working... done
Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.9.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0


julia(10930) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


PyObject <function roc_auc_score at 0x27ec5f1c0>

In [263]:
policy_outcomes = IAI.predict_outcomes(grid, test_X, test_rewards)
abs_policy_outcomes = abs.(policy_outcomes)

4807-element Vector{Float64}:
  12.97303046682081
   5.185130724389097
   6.978810416269162
   3.4272578999389838
 147.35312560612329
  10.986964824374558
   9.86427625808887
   5.443185643261696
  20.521586750953524
   3.105876334803343
  26.119571652050944
   9.116016864779636
  24.374684927745157
   ⋮
  11.914105560487574
  16.378113722230278
  37.24034794540616
  13.990443250457318
   6.379381960787546
  10.845049262086151
  24.040287971260522
  38.82488048541673
  13.994765495094011
  32.26793812833023
   7.171837840923153
   6.352049336426518

In [318]:
using Statistics
mean(policy_outcomes)

10.4451646847752

In [272]:
##### baseline
mean(test_rewards[:, "75"])

21.61558747813238